In [1]:
import pandas as pd
import numpy as np
data=pd.read_csv("C:\\Users\\91852\\Downloads\\ParsedLogs.csv")
data.head()

,"id,message,@version,@timestamp"
0,"3y51c38BQ2yDyGVTnYjI,kernel audit,1,2022-03-10..."
1,"4C51c38BQ2yDyGVTnYjI,kernel,1,2022-03-10T10:52..."
2,"4S51c38BQ2yDyGVTnYjI,kernel Extensions device ..."
3,"4i51c38BQ2yDyGVTnYjI,kernel tunneling driver,1..."
4,"4y51c38BQ2yDyGVTnYjI,kernel audit,1,2022-03-10..."


In [2]:
df=data['id,message,@version,@timestamp'].str.split(',',expand=True)

In [3]:
df=df[1]

In [4]:
df.head()

0                     kernel audit
1                           kernel
2    kernel Extensions device dblo
3          kernel tunneling driver
4                     kernel audit
Name: 1, dtype: object

In [5]:
df=df.to_frame()

In [6]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25567 entries, 0 to 25566
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   1       25567 non-null  object
dtypes: object(1)
memory usage: 199.9+ KB


In [7]:
df[1]

0                                kernel audit
1                                      kernel
2               kernel Extensions device dblo
3                     kernel tunneling driver
4                                kernel audit
                         ...                 
25562                                  kernel
25563                                  kernel
25564         kernel checking image initramfs
25565    kernel Freeing initrd memory k freed
25566               kernel Autodetecting RAID
Name: 1, Length: 25567, dtype: object

In [8]:
messages=df

In [9]:
corpus=set()

In [10]:
for str in df[1]:
    corpus.add(str)

In [11]:
corpus

{'',
 'FreeWnn jserver shutdown failed',
 'FreeWnn jserver shutdown succeeded',
 'IIim htt succeeded',
 'PAMrootok pam_check_access failed proper access system_usystem_rkernel_t',
 'anacron anacron succeeded',
 'apmd Charge unknown',
 'apmd Exiting',
 'apmd Version',
 'apmd apmd shutdown succeeded',
 'apmd apmd succeeded',
 'atd atd shutdown succeeded',
 'atd atd succeeded',
 'autofs automount succeeded',
 'bluetooth hcid succeeded',
 'bluetooth sdpd succeeded',
 'canna Stopping Canna server succeeded',
 'canna succeeded',
 'cannaserver irw_killservercannaserver end',
 'crond crond shutdown succeeded',
 'crond crond succeeded',
 'cup cupsd shutdown succeeded',
 'cup cupsd succeeded',
 'cupslpd Connection cable',
 'cupslpd Unable get command line client',
 'dd record',
 'exiting signal',
 'fingerd Client hung probable portscan',
 'ftpd ANONYMOUS FTP LOGIN FROM',
 'ftpd User unknown timed second',
 'ftpd connection',
 'ftpd getpeername ftpd Transport endpoint connected',
 'ftpd setsockop

In [12]:
words = []
for text in corpus:
    for word in text.split(' '):
        words.append(word)

words = set(words)
words=list(words)

In [13]:
words

['',
 'recovery',
 'handle_mm_faultxdxe',
 'rhostn',
 'net',
 'fast',
 'architecture',
 'aecc',
 'idebusxx',
 'zapping',
 'jbd',
 'Busy',
 'removing',
 'support',
 'common',
 'udev',
 'proc',
 'ffb',
 'controller',
 'spurious',
 'broadcasted',
 'DMI',
 'probable',
 'memory',
 'Bringing',
 'Address',
 'filesystems',
 'waiting',
 'mrtg',
 'journal_startxxe',
 'tty',
 'grow_dev_pagexxc',
 'cpu',
 'relabel',
 'TERM',
 'deleted',
 'Exiting',
 'USB',
 'PC',
 'devhda',
 'connection',
 'permissive',
 'isapnp',
 'Scaffold',
 'python',
 'get_user_sizexx',
 'cf',
 'call_console_driversxbexe',
 'driver',
 'unmap_vma_listxex',
 'A',
 'runtime',
 'main',
 'terminating',
 'xinetd',
 'addb',
 'existing',
 'serio',
 'bridge',
 'API',
 'dee',
 'smux_accept',
 'filemap_nopagexxd',
 'rhostjwhp',
 'Support',
 'ROOT',
 'Linux',
 'logrotate',
 'sysctl',
 'parameter',
 'agp',
 'precision',
 'AGP',
 'clear_page_tablesxcxf',
 'superblocks',
 'forked',
 'rhostarx',
 'STOP',
 'channel',
 'sbinmingetty',
 'smartd'

In [14]:
len(words)

928

## Data Generation

### Creating input and label for skip gram

In [15]:
word2int = {}

for i,word in enumerate(words):
    word2int[word] = i

sentences = []
for sentence in corpus:
    sentences.append(sentence.split())
    
WINDOW_SIZE =4 

data = []
for sentence in sentences:
    for idx, word in enumerate(sentence):
        for neighbor in sentence[max(idx - WINDOW_SIZE, 0) : min(idx + WINDOW_SIZE, len(sentence)) + 1] : 
            if neighbor != word:
                data.append([word, neighbor])

In [16]:
df = pd.DataFrame(data, columns = ['input', 'label'])

In [17]:
df.head(10)

,input,label
0,supam_unix,session
1,supam_unix,closed
2,session,supam_unix
3,session,closed
4,closed,supam_unix
5,closed,session
6,kernel,audit
7,kernel,initializing
8,kernel,netlink
9,kernel,socket


In [18]:
df.shape

(8564, 2)

## Define tensorflow graph

In [19]:
ONE_HOT_DIM = len(words)

# function to convert numbers to one hot vectors
def to_one_hot_encoding(data_point_index):
    one_hot_encoding = np.zeros(ONE_HOT_DIM)
    one_hot_encoding[data_point_index] = 1
    return one_hot_encoding

X = [] # input word
Y = [] # target word

for x, y in zip(df['input'], df['label']):
    X.append(to_one_hot_encoding(word2int[ x ]))
    Y.append(to_one_hot_encoding(word2int[ y ]))


In [20]:
# convert them to numpy arrays
X_train = np.asarray(X)
Y_train = np.asarray(Y)


In [21]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
# making placeholders for X_train and Y_train
x = tf.placeholder(tf.float32, shape=(None, ONE_HOT_DIM))
y_label = tf.placeholder(tf.float32, shape=(None, ONE_HOT_DIM))

Instructions for updating:
non-resource variables are not supported in the long term


In [22]:
EMBEDDING_DIM = 8

In [23]:
# hidden layer: which represents word vector eventually
W1 = tf.Variable(tf.random_normal([ONE_HOT_DIM, EMBEDDING_DIM]))
b1 = tf.Variable(tf.random_normal([1])) #bias
hidden_layer = tf.add(tf.matmul(x,W1), b1)

# output layer
W2 = tf.Variable(tf.random_normal([EMBEDDING_DIM, ONE_HOT_DIM]))
b2 = tf.Variable(tf.random_normal([1]))
prediction = tf.nn.softmax(tf.add( tf.matmul(hidden_layer, W2), b2))

# loss function: cross entropy
loss = tf.reduce_mean(-tf.reduce_sum(y_label * tf.log(prediction), axis=[1]))

# training operation
train_op = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

## Train

In [27]:
import time

In [28]:
st_time=time.time()
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init) 

iteration = 500
for i in range(iteration):
    # input is X_train which is one hot encoded word
    # label is Y_train which is one hot encoded neighbor word
    sess.run(train_op, feed_dict={x: X_train, y_label: Y_train})
    if i % 25 == 0:
        print('iteration '+' loss is : ', sess.run(loss, feed_dict={x: X_train, y_label: Y_train}))
end_time=time.time()

iteration  loss is :  10.84976
iteration  loss is :  10.433709
iteration  loss is :  10.287796
iteration  loss is :  10.163575
iteration  loss is :  10.054743
iteration  loss is :  9.957936
iteration  loss is :  9.871043
iteration  loss is :  9.792559
iteration  loss is :  9.721294
iteration  loss is :  9.656262
iteration  loss is :  9.596617
iteration  loss is :  9.541621
iteration  loss is :  9.490645
iteration  loss is :  9.443148
iteration  loss is :  9.3986635
iteration  loss is :  9.35681
iteration  loss is :  9.31726
iteration  loss is :  9.279741
iteration  loss is :  9.24403
iteration  loss is :  9.209933


In [29]:
print(end_time-st_time)

80.43895888328552


In [30]:
# Now the hidden layer (W1 + b1) is actually the word look up table
vectors = sess.run(W1 + b1)
print(vectors)

[[-1.2853861   0.09649899  1.4555501  ... -2.6598277   0.03206141
  -1.6277922 ]
 [-0.24491492  1.835547    1.2525043  ...  0.9402367  -0.02381339
  -0.49135715]
 [ 0.23639509 -0.620435   -1.5652677  ...  0.8457012  -2.4497828
  -1.3569902 ]
 ...
 [ 1.3179003  -0.63662136  0.55702835 ...  0.48317292  0.7395158
   0.42405647]
 [ 0.72877586 -0.5119296  -1.8720145  ...  0.9964946   0.6931098
   0.6037278 ]
 [ 1.0636101   2.4852855   0.57719976 ... -0.23530868  0.48109344
  -0.26109818]]


## Word Vector table

In [31]:
vec = pd.DataFrame(vectors)

In [32]:
vec

,0,1,2,3,4,5,6,7
0,-1.285386,0.096499,1.455550,0.896479,0.575843,-2.659828,0.032061,-1.627792
1,-0.244915,1.835547,1.252504,-0.242595,0.907250,0.940237,-0.023813,-0.491357
2,0.236395,-0.620435,-1.565268,-0.649701,1.460847,0.845701,-2.449783,-1.356990
3,0.126449,1.242163,0.847720,0.398146,-0.783777,-0.392336,0.529852,1.368142
4,-0.353036,1.815627,-1.223340,0.417484,0.839034,0.594658,-0.820962,0.967912
...,...,...,...,...,...,...,...,...
923,-0.354510,-1.385375,1.491804,-0.112686,0.982850,-0.176440,-0.514864,-0.244191
924,-1.332364,-0.777031,-0.915395,0.092365,-1.035571,0.163366,0.687981,-0.797913
925,1.317900,-0.636621,0.557028,-0.856483,0.643806,0.483173,0.739516,0.424056
926,0.728776,-0.511930,-1.872015,0.957370,-0.126739,0.996495,0.693110,0.603728


# Preapring input for clustering

In [33]:
messages.head()

,1
0,kernel audit
1,kernel
2,kernel Extensions device dblo
3,kernel tunneling driver
4,kernel audit


In [34]:
messages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25567 entries, 0 to 25566
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   1       25567 non-null  object
dtypes: object(1)
memory usage: 199.9+ KB


In [35]:
sentences=[]
for str in messages[1]:
    sentences.append(str)

In [36]:
type(sentences[0])

str

In [37]:
print(type(words))
l=len(words)
print(l)

<class 'list'>
928


In [38]:
dict_words={}

In [39]:
for i in range(l):
    dict_words[words[i]]=list(vec.iloc[i,:])


In [40]:
print(type(dict_words["end"]))

<class 'list'>


### Calculating mean of vectors

In [41]:
list_final=[]
for text in sentences:
    temp=text.split(' ')
    lt=[]
    for s in temp:
        lt.append(dict_words[s])
    npat=np.array(lt)
    t=np.mean(npat,axis=0).tolist()
    list_final.append(t)

In [42]:
list_final

[[-0.16793562471866608,
  -0.005547918379306793,
  0.14906006585806608,
  -0.1852780357003212,
  -0.6205652058124542,
  0.6495263613760471,
  0.6209213472902775,
  -0.3428526446223259],
 [0.13903182744979858,
  -0.1757565140724182,
  0.0047100987285375595,
  -0.2186027318239212,
  -0.18212038278579712,
  -0.08601485937833786,
  0.05579967051744461,
  0.015620782971382141],
 [-0.1983000934123993,
  -0.5710196923464537,
  0.34441923862323165,
  -0.1132304621860385,
  -0.26925235241651535,
  0.27836343087255955,
  0.22746254689991474,
  0.70383127219975],
 [0.5681431194146475,
  0.4696666399637858,
  1.0155138242989779,
  0.045737547179063164,
  -0.31712238987286884,
  -0.5654477551579475,
  -0.1607837105790774,
  -0.8212515066067377],
 [-0.16793562471866608,
  -0.005547918379306793,
  0.14906006585806608,
  -0.1852780357003212,
  -0.6205652058124542,
  0.6495263613760471,
  0.6209213472902775,
  -0.3428526446223259],
 [-0.16793562471866608,
  -0.005547918379306793,
  0.14906006585806608,

In [43]:
cl_input=pd.DataFrame(list_final)

In [46]:
cl_input

,0,1,2,3,4,5,6,7
0,-0.167936,-0.005548,0.149060,-0.185278,-0.620565,0.649526,0.620921,-0.342853
1,0.139032,-0.175757,0.004710,-0.218603,-0.182120,-0.086015,0.055800,0.015621
2,-0.198300,-0.571020,0.344419,-0.113230,-0.269252,0.278363,0.227463,0.703831
3,0.568143,0.469667,1.015514,0.045738,-0.317122,-0.565448,-0.160784,-0.821252
4,-0.167936,-0.005548,0.149060,-0.185278,-0.620565,0.649526,0.620921,-0.342853
...,...,...,...,...,...,...,...,...
25562,0.139032,-0.175757,0.004710,-0.218603,-0.182120,-0.086015,0.055800,0.015621
25563,0.139032,-0.175757,0.004710,-0.218603,-0.182120,-0.086015,0.055800,0.015621
25564,-0.116574,0.125870,0.143373,-0.170841,-0.204182,1.078701,0.603832,-0.544247
25565,0.312183,-0.127498,-0.186760,-0.321333,-0.223586,0.148211,0.285270,0.098700


In [49]:
cl_input.to_csv(r'C:\Users\91852\OneDrive\Desktop\cl_input.csv', index=False)